In [1]:
# Example of training a network by backpropagation
from math import exp
from random import seed
from random import random
from random import randrange
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# genertae toy dataset
def generate_data (num_sample_class, dim, num_class):
    X = np.zeros((num_sample_class*num_class,dim)) # data matrix (each row = single example)
    y = np.zeros(num_sample_class*num_class, dtype='uint8') # class labels
    for j in range(num_class):
        ix = range(num_sample_class*j,num_sample_class*(j+1))
        r = np.linspace(0.0,1,num_sample_class) # radius
        t = np.linspace(j*4,(j+1)*4,num_sample_class) + np.random.randn(num_sample_class)*0.2 # theta
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        y[ix] = j
    dataset = list()
    for i in range(len(X)):
        lst = X[i].tolist()
        lst.append(y[i])
        dataset.append(lst)
    # lets visualize the data:
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
    plt.show()
    return (dataset)

In [3]:
# Split a dataset into a train and test set
def train_test_split(dataset, split):
	train = list()
	train_size = split * len(dataset)
	dataset_copy = list(dataset)
	while len(train) < train_size:
		index = randrange(len(dataset_copy))
		train.append(dataset_copy.pop(index))
	return train, dataset_copy

In [ ]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]            # to get the bias. If you dont have bias then set the activation to zero
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]  # sum up the product of the weight and input value
    return activation

# Transfer neuron activation using sigmoid function
from math import exp
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward-propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']


# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    return(network)

In [ ]:
# train/test split
seed(1)

N = 100 # number of samples per class
D = 2 # dimensionality
K = 3 # number of classes

dataset  =  generate_data (N, D, K)


# normalize input variables
#minmax = dataset_minmax(dataset)
#normalize_dataset(dataset, minmax)

train, test = train_test_split(dataset, split = 0.8)

# training parameters
l_rate = 0.1
n_epoch = 1000
n_hidden = 20

# train the network
Model = back_propagation(train, l_rate, n_epoch, n_hidden)

#make prediction on the test set
PredClass = list()
ActualClass = list()
for row in test:
    prediction = predict(Model, row)
    PredClass.append(prediction)
    ActualClass.append(row[-1])
    print('Expected=%d, Got=%d' % (row[-1], prediction))
    
accuracy = accuracy_metric(ActualClass, PredClass)
accuracy